[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/UM-RMRS/raster_tools/blob/main/notebooks/SampleDesign.ipynb)

# Sample Design App
## This app will allow you to create random, systematic, and tiled sample designs.
### Steps
1. Draw a polygon on the map that spatially identifies the boundary of the study unit.
2. select the sample type and design values.
4. Click the run button. Sample point locations will be added to the map and will be zipped up for download. 

In [ ]:
import smpDsg
import ipyleaflet
import ipywidgets
import localtileserver
import geopandas as gpd
import shapely
import base64
import os

if('LOCALTILESERVER_CLIENT_PREFIX' in os.environ.keys()):
    os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = f"{os.environ['JUPYTERHUB_SERVICE_PREFIX']}/proxy/{{port}}"

ly = ipywidgets.Layout(
    height='700px',
    )

lc = ipyleaflet.LayersControl()
lc.position='topright'
dr=ipyleaflet.DrawControl()
dr.position='topleft'


bmaps=ipyleaflet.basemaps

m=ipyleaflet.Map(controls=[lc,dr],scroll_wheel_zoom=True,center=(41,-119),zoom=5,layout=ly)

tly1=ipyleaflet.basemap_to_tiles(bmaps.Esri.WorldImagery)
tly1.base = False
tly1.name = 'World Imagery'

tly2=ipyleaflet.ImageService(
    url=r'https://lfps.usgs.gov/arcgis/rest/services/Landfire_LF230/US_230EVT/ImageServer',
    format='jpgpng',
    attribution='USDA USFS',
    name = 'Landfire EVT',
)


tly3=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/Nat_BurnProbability/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name = 'Burn Probability'
)

tly4=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/Potential_Control_Location/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name='PCL',
)
tly5=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/SnagHazard_eDart/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name='Snag Hazard eDart',
)
tly6=ipyleaflet.ImageService(
    url=r'https://apps.fs.usda.gov/fsgisx03/rest/services/wo_spf_fam/Suppression_Difficulty_Index_80/ImageServer',
    format='jpgpng',
    attribution='USDA FAM',
    name='SDI 80',

)

baselyrDic = {
    'World Imagery':tly1,
    'Landfire EVT': tly2,
    'Burn Probability': tly3,
    'PCL': tly4,
    'Snag Hazard eDart':tly5,
    'SDI 80':tly6,
}

def add_remove_layer(b):
    nm = b['owner'].description
    lyr = baselyrDic[nm]
    if b['new']:
        m.add(lyr)
    else:
        m.remove(lyr)


#widgets
style = {'description_width': 'initial'}

#optional background layers
l1_o=ipywidgets.widgets.Checkbox(value=False,description='World Imagery', disabled=False, indent=False)
l1_o.observe(add_remove_layer,names='value')
l2_o=ipywidgets.widgets.Checkbox(value=False,description='Landfire EVT', disabled=False, indent=False)
l2_o.observe(add_remove_layer,names='value')
l3_o=ipywidgets.widgets.Checkbox(value=False,description='Burn Probability', disabled=False, indent=False)
l3_o.observe(add_remove_layer,names='value')
l4_o=ipywidgets.widgets.Checkbox(value=False,description='PCL', disabled=False, indent=False)
l4_o.observe(add_remove_layer,names='value')
l5_o=ipywidgets.widgets.Checkbox(value=False,description='Snag Hazard eDart', disabled=False, indent=False)
l5_o.observe(add_remove_layer,names='value')
l6_o=ipywidgets.widgets.Checkbox(value=False,description='SDI 80', disabled=False, indent=False)
l6_o.observe(add_remove_layer,names='value')

thb=ipywidgets.HBox([l1_o,l2_o,l3_o,l4_o,l5_o,l6_o])  

lbl=ipywidgets.Label(value='Add/Remove Background Layers')
lbl.style.font_size="20px"
thb=ipywidgets.VBox([lbl,thb])

def add_widgets(b):
    vl=b['new']
    if(vl=='systematic'):
        wb.children=tuple(list(wb.children[:3]) +[smp_spcx,smp_spcy,btn])
    elif(vl=='stratified'):
        wb.children=tuple(list(wb.children[:3]) +[smp_pnt,smp_rsp,smp_date1,smp_date2,btn])
    else:
        wb.children=tuple(list(wb.children[:3]) +[smp_pnt,btn])


#widgets
style = {
    'description_width': 'initial',
    'width': 'max-content',
    }

#main widgets
smp_dsg=ipywidgets.widgets.Dropdown(
    options=['random', 'systematic','stratified'],
    value='random',
    description='Sample Type:',
    disabled=False,
    style=style,
)
smp_dsg.observe(add_widgets,names='value')

smp_pnt=ipywidgets.widgets.IntText(
    value=30,
    description='Sample Size',
    disable=False,
    style=style,
)
smp_spcx=ipywidgets.widgets.IntText(
    value=2,
    description='Distance x (m)',
    disable=False,
    style=style,
)
smp_spcy=ipywidgets.widgets.IntText(
    value=2,
    description='Distance y (m)',
    disable=False,
    style=style,

)

smp_rsp=ipywidgets.widgets.SelectMultiple(
    options=['elevation', 'sentinel2', 'lidar','landsat'],
    value=['elevation'],
    description='Select spreading sources:',
    disabled=False,

    style=style,
)
smp_date1=ipywidgets.widgets.DatePicker(
    description='Start date',
    disable=False,
    style=style,
)
smp_date2=ipywidgets.widgets.DatePicker(
    description='End date',
    disable=False,
    style=style,
)

prg_wdg=ipywidgets.widgets.IntProgress(value=0,min=0,max=12,description='Progress',style={'bar_color':'green'},orientation='horizontal')
out_wdg=ipywidgets.widgets.Output(layout={'border': '1px solid black'})

def create_download_link(filename, title = "Click here to download: "):  
    fl = open(filename, "rb")
    data = fl.read()
    b64 = base64.b64encode(data)
    payload = b64.decode()
    fl.close()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title+f' {filename}',filename=filename)
    return html

cnt=1

def _run_sample_design(b):
    global cnt
    
    polylst=[]
    for d in dr.data:
        geo=d['geometry']
        t=geo['type']
        if(t=='Polygon'):polylst.append(geo['coordinates'][0])

    if((len(polylst)<1)):
        print("You Must Create a Study Area afore clicking the Run button")
        return
    else:
        geo=gpd.GeoSeries(shapely.polygons(polylst),crs=4326)
        geop=geo.to_crs(5070)
        print('Projected to Albers Equal Area ...')
        print('Creating Sample ...')
        prg_wdg.value=5

        gdfp=None
        if(smp_dsg.value=='systematic'):
            gdfp=smpDsg.get_systematic_sample(geop,smp_spcx.value,smp_spcy.value,) 
        elif(smp_dsg.value=='random'):
            gdfp=smpDsg.get_random_sample(geop,smp_pnt.value)
        else:
            gdfp=smpDsg.get_random_sample(geop,smp_pnt.value)
        
        gdfg=gdfp.to_crs(4326)
        fnm=smp_dsg.value + 'sample ' + str(cnt)
        print('Adding sample to the map ...')
        prg_wdg.value=10

        lyr=ipyleaflet.GeoData(geo_dataframe=gdfg,
                               style={'color': 'yellow', 'radius':4, 'fillColor': 'yellow', 'opacity':1, 'weight':2, 'dashArray':'2', 'fillOpacity':1},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               point_style={'radius': 4, 'color': 'yellow', 'fillOpacity': 1, 'fillColor': 'yellow', 'weight': 3},
                               name = fnm)
        
        print('Creating download file ...')
        gdfp.to_file(fnm + '.shp.zip',driver='ESRI Shapefile')
        html=create_download_link(fnm+'.shp.zip')
        wb.children=tuple(list(wb.children) +[ipywidgets.HTML(html)])

        m.add(lyr)
        cnt+=1
        prg_wdg.value=prg_wdg.max
        print('Finished all processing ...')

        return gdfp

@out_wdg.capture()
def sample_design(b):
    prg_wdg.value=0
    out_wdg.clear_output()

  
    #check values
    gdf=_run_sample_design(b)
        
    return gdf

  
btn=ipywidgets.widgets.Button(description='Run')
gdf=btn.on_click(_run_sample_design)


wb=ipywidgets.VBox([ipywidgets.widgets.Label(value=""),ipywidgets.widgets.Label(value=""),smp_dsg,smp_pnt,btn],layout=ipywidgets.Layout(width='32%'))

rb=ipywidgets.VBox([thb,m])

mb=ipywidgets.HBox([wb,rb])

ipywidgets.VBox([mb,prg_wdg,out_wdg])